In [1]:
#!pip install polygon-api-client

#!pip install tqdm

#!pip install ipywidgets widgetsnbextension pandas-profiling


In [3]:
apikey = "wBkWOy90budzobSlu_fSbwiTrf9JVb20"

In [69]:
from polygon import RESTClient
from local_settings import polygon as settings
from tqdm.notebook import tqdm
from joblib import Parallel,delayed

In [5]:
import pickle

def save(l,filename):
    with open(filename, 'wb') as f:
         pickle.dump(l, f)
            
            
def load(filename):
    with open(filename, 'rb') as g:
         l = pickle.load(g)
    return l

In [6]:
from datetime import date, datetime
from typing import Any, Optional
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

markets = ['crypto', 'stocks', 'fx']

class MyRESTClient(RESTClient):
    def __init__(self, auth_key: str=settings['api_key'], timeout:int=5):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10,
                               backoff_factor=10,
                               status_forcelist=[429, 500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        #self._session.mount('https://', adapter)

    def get_tickers(self, market:str=None) -> pd.DataFrame:
        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        resp = self.reference_tickers_v3(market=market)
        if hasattr(resp, 'results'):
            df = pd.DataFrame(resp.results)

            while hasattr(resp, 'next_url'):
                resp = self.reference_tickers_v3(next_url=resp.next_url)
                df = df.append(pd.DataFrame(resp.results))

            if market == 'crypto':
                df = df[df['currency_symbol'] == 'USD']
                df['name'] = df['base_currency_name']
                df = df[['ticker', 'name', 'market', 'active']]

            df = df.drop_duplicates(subset='ticker')
            return df
        return None
        
        

In [7]:
client = MyRESTClient(settings['api_key'])

In [8]:
#df = client.get_tickers('stocks')

In [10]:
ticker = "AAPL"

# List Aggregates (Bars)
bars = client.get_aggs(ticker=ticker, multiplier=1, timespan="day", from_="2023-01-09", to="2023-01-10")
for bar in bars:
    print(bar)


Agg(open=130.465, high=133.41, low=129.89, close=130.15, volume=70790813.0, vwap=131.6292, timestamp=1673240400000, transactions=645365, otc=None)
Agg(open=130.26, high=131.2636, low=128.12, close=130.73, volume=63896155.0, vwap=129.822, timestamp=1673326800000, transactions=554940, otc=None)


In [19]:
bars[0].timestamp

1673240400000

In [23]:
pd.Timestamp(bars[0].timestamp, unit = 'ms')

Timestamp('2023-01-09 05:00:00')

In [25]:
pd.Timestamp(bars[1].timestamp, unit = 'ms')

Timestamp('2023-01-10 05:00:00')

In [7]:
exchanges = client.get_exchanges()

In [10]:
exchanges

[Exchange(acronym='AMEX', asset_class='stocks', id=1, locale='us', mic='XASE', name='NYSE American, LLC', operating_mic='XNYS', participant_id='A', type='exchange', url='https://www.nyse.com/markets/nyse-american'),
 Exchange(acronym=None, asset_class='stocks', id=2, locale='us', mic='XBOS', name='Nasdaq OMX BX, Inc.', operating_mic='XNAS', participant_id='B', type='exchange', url='https://www.nasdaq.com/solutions/nasdaq-bx-stock-market'),
 Exchange(acronym='NSX', asset_class='stocks', id=3, locale='us', mic='XCIS', name='NYSE National, Inc.', operating_mic='XNYS', participant_id='C', type='exchange', url='https://www.nyse.com/markets/nyse-national'),
 Exchange(acronym=None, asset_class='stocks', id=4, locale='us', mic='FINY', name='FINRA NYSE TRF', operating_mic='XNYS', participant_id='D', type='TRF', url='https://www.finra.org'),
 Exchange(acronym=None, asset_class='stocks', id=4, locale='us', mic='FINN', name='FINRA Nasdaq TRF Carteret', operating_mic='FINR', participant_id='D', typ

In [66]:
from polygon import RESTClient

# docs
# https://polygon.io/docs/stocks/get_v3_reference_tickers
# https://polygon-api-client.readthedocs.io/en/latest/Reference.html#list-tickers

# client = RESTClient("XXXXXX") # hardcoded api_key is used
client = RESTClient(apikey)  # POLYGON_API_KEY environment variable is used

#tickers = []
#for t in tqdm(client.list_tickers()):
#    tickers.append(t)
#print(tickers)

In [69]:
tickers = load('tickers')

In [84]:
t = tickers[0]

In [60]:
save(tickers,'tickers')

In [23]:
financials  = client.get_ticker_details(t.ticker)

In [43]:
d = {}

for k in financials.__dict__.keys():
    d[k] = []
        

In [11]:
#for t in tqdm(tickers):
#    tmp = client.get_ticker_details(t.ticker).__dict__
#    for k in d.keys():
#        d[k].append(tmp[k])

In [86]:
def get_financials_as_dict(t):
    financials  = client.get_ticker_details(t.ticker).__dict__
    return financials

In [88]:
client.get_ticker_details(t.ticker).__dict__

{'active': True,
 'address': CompanyAddress(address1='5301 STEVENS CREEK BLVD', address2=None, city='SANTA CLARA', state='CA', country=None, postal_code='95051'),
 'branding': Branding(icon_url='https://api.polygon.io/v1/reference/company-branding/d3d3LmFnaWxlbnQuY29t/images/2023-03-01_icon.jpeg', logo_url='https://api.polygon.io/v1/reference/company-branding/d3d3LmFnaWxlbnQuY29t/images/2023-03-01_logo.svg', accent_color=None, light_color=None, dark_color=None),
 'cik': '0001090872',
 'composite_figi': 'BBG000C2V3D6',
 'currency_name': 'usd',
 'currency_symbol': None,
 'base_currency_name': None,
 'base_currency_symbol': None,
 'delisted_utc': None,
 'description': "Originally spun out of Hewlett-Packard in 1999, Agilent has evolved into a leading life sciences and diagnostics firm. Today, Agilent's measurement technologies serve a broad base of customers with its three operating segments: life science and applied tools, cross lab (consisting of consumables and services related to its 

In [89]:
#parallel = Parallel(n_jobs = -1, verbose = True)

#all_financials = parallel([delayed(get_financials_as_dict)(t) for t in tickers])

In [62]:
types = client.get_ticker_types()
print(types)

[TickerTypes(asset_class='stocks', code='CS', description='Common Stock', locale='us'), TickerTypes(asset_class='stocks', code='PFD', description='Preferred Stock', locale='us'), TickerTypes(asset_class='stocks', code='WARRANT', description='Warrant', locale='us'), TickerTypes(asset_class='stocks', code='RIGHT', description='Rights', locale='us'), TickerTypes(asset_class='stocks', code='BOND', description='Corporate Bond', locale='us'), TickerTypes(asset_class='stocks', code='ETF', description='Exchange Traded Fund', locale='us'), TickerTypes(asset_class='stocks', code='ETN', description='Exchange Traded Note', locale='us'), TickerTypes(asset_class='stocks', code='ETV', description='Exchange Traded Vehicle', locale='us'), TickerTypes(asset_class='stocks', code='SP', description='Structured Product', locale='us'), TickerTypes(asset_class='stocks', code='ADRC', description='American Depository Receipt Common', locale='us'), TickerTypes(asset_class='stocks', code='ADRP', description='Amer

In [13]:
trades = []
for t in tqdm(client.list_trades("A", "2022-02-01", limit=50000)):
    trades.append(t)

0it [00:00, ?it/s]

In [16]:
trades[0].__dict__

{'conditions': [12],
 'exchange': 11,
 'id': '52983577858113',
 'participant_timestamp': 1643760512678792704,
 'price': 141,
 'sequence_number': 2587171,
 'sip_timestamp': 1643760512678833664,
 'size': 100,
 'tape': 1}

In [74]:
financials = []
for f in client.vx.list_stock_financials("AAPL"):
    financials.append(f)
print(financials)

[StockFinancial(cik='0000320193', company_name='APPLE INC', end_date='2022-12-31', filing_date=None, financials=Financials(balance_sheet={'equity': DataPoint(formula=None, label='Equity', order=1400, unit='USD', value=56727000000.0, xpath=None), 'equity_attributable_to_noncontrolling_interest': DataPoint(formula=None, label='Equity Attributable To Noncontrolling Interest', order=1500, unit='USD', value=0, xpath=None), 'equity_attributable_to_parent': DataPoint(formula=None, label='Equity Attributable To Parent', order=1600, unit='USD', value=56727000000.0, xpath=None), 'noncurrent_assets': DataPoint(formula=None, label='Noncurrent Assets', order=300, unit='USD', value=217970000000.0, xpath=None), 'liabilities': DataPoint(formula=None, label='Liabilities', order=600, unit='USD', value=290020000000.0, xpath=None), 'assets': DataPoint(formula=None, label='Assets', order=100, unit='USD', value=346747000000.0, xpath=None), 'current_assets': DataPoint(formula=None, label='Current Assets', or

In [83]:
financials[0]

StockFinancial(cik='0000320193', company_name='APPLE INC', end_date='2022-12-31', filing_date=None, financials=Financials(balance_sheet={'equity': DataPoint(formula=None, label='Equity', order=1400, unit='USD', value=56727000000.0, xpath=None), 'equity_attributable_to_noncontrolling_interest': DataPoint(formula=None, label='Equity Attributable To Noncontrolling Interest', order=1500, unit='USD', value=0, xpath=None), 'equity_attributable_to_parent': DataPoint(formula=None, label='Equity Attributable To Parent', order=1600, unit='USD', value=56727000000.0, xpath=None), 'noncurrent_assets': DataPoint(formula=None, label='Noncurrent Assets', order=300, unit='USD', value=217970000000.0, xpath=None), 'liabilities': DataPoint(formula=None, label='Liabilities', order=600, unit='USD', value=290020000000.0, xpath=None), 'assets': DataPoint(formula=None, label='Assets', order=100, unit='USD', value=346747000000.0, xpath=None), 'current_assets': DataPoint(formula=None, label='Current Assets', ord

In [18]:
# stocks * days * avg num trades

1000*1000*30000

30000000000

In [67]:
aggs = client.get_aggs(
    "AAPL",
    1,
    "minute",
    "1675036800000",
    "1679270400000",
)

SyntaxError: invalid syntax (123086888.py, line 5)

In [66]:
len(aggs)

5000

In [52]:
l = [x.timestamp for x in aggs]

In [55]:
24*60

1440

In [64]:
(1679270400000 - 1675036800000)/(1000*60)

70560.0

In [56]:
s = pd.Timestamp("2023-01-30")

In [57]:
r = s.timestamp()

In [50]:
pd.Timestamp(1679270400000,unit = 'ms')

Timestamp('2023-03-20 00:00:00')

In [58]:
r

1675036800.0

In [90]:
import os

d1 = load('./financials_dict')

In [92]:
len(set(d2['ticker']))

13000

In [91]:
len(set(d2['ticker'] + d1['ticker']))

13000

In [94]:
d1['ticker']

['A',
 'AA',
 'AAA',
 'AAAIF',
 'AAALF',
 'AAALY',
 'AAAU',
 'AABB',
 'AABVF',
 'AAC',
 'AAC.U',
 'AAC.WS',
 'AACAF',
 'AACAY',
 'AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AACS',
 'AACTF',
 'AADI',
 'AADR',
 'AAEEF',
 'AAFRF',
 'AAGC',
 'AAGEF',
 'AAGFF',
 'AAGH',
 'AAGIY',
 'AAGRY',
 'AAHIF',
 'AAIC',
 'AAICpB',
 'AAICpC',
 'AAIGF',
 'AAIIQ',
 'AAIN',
 'AAIRF',
 'AAL',
 'AALBF',
 'AAMAF',
 'AAMC',
 'AAME',
 'AAMMF',
 'AAMTF',
 'AAMpA',
 'AAMpB',
 'AAN',
 'AANNF',
 'AAOI',
 'AAON',
 'AAP',
 'AAPB',
 'AAPD',
 'AAPJ',
 'AAPL',
 'AAPT',
 'AAPU',
 'AARTY',
 'AASCF',
 'AASL',
 'AASP',
 'AASZF',
 'AAT',
 'A',
 'AA',
 'AAA',
 'AAAIF',
 'AAALF',
 'AAALY',
 'AAAU',
 'AABB',
 'AABVF',
 'AAC',
 'AAC.U',
 'AAC.WS',
 'AACAF',
 'AACAY',
 'AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AACS',
 'AACTF',
 'AADI',
 'AADR',
 'A',
 'AA',
 'AAA',
 'AAAIF',
 'AAALF',
 'AAALY',
 'AAAU',
 'AABB',
 'AABVF',
 'AAC',
 'AAC.U',
 'AAC.WS',
 'AACAF',
 'AACAY',
 'AACG',
 'A',
 'AA',
 'AAA',
 'AAAIF',
 'AAALF',
 'AAALY',
 'AAAU',

In [93]:
d2['ticker']

['A',
 'AA',
 'AAA',
 'AAAIF',
 'AAALF',
 'AAALY',
 'AAAU',
 'AABB',
 'AABVF',
 'AAC',
 'AAC.U',
 'AAC.WS',
 'AACAF',
 'AACAY',
 'AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AACS',
 'AACTF',
 'AADI',
 'AADR',
 'AAEEF',
 'AAFRF',
 'AAGC',
 'AAGEF',
 'AAGFF',
 'AAGH',
 'AAGIY',
 'AAGRY',
 'AAHIF',
 'AAIC',
 'AAICpB',
 'AAICpC',
 'AAIGF',
 'AAIIQ',
 'AAIN',
 'AAIRF',
 'AAL',
 'AALBF',
 'AAMAF',
 'AAMC',
 'AAME',
 'AAMMF',
 'AAMTF',
 'AAMpA',
 'AAMpB',
 'AAN',
 'AANNF',
 'AAOI',
 'AAON',
 'AAP',
 'AAPB',
 'AAPD',
 'AAPJ',
 'AAPL',
 'AAPT',
 'AAPU',
 'AARTY',
 'AASCF',
 'AASL',
 'AASP',
 'AASZF',
 'AAT',
 'A',
 'AA',
 'AAA',
 'AAAIF',
 'AAALF',
 'AAALY',
 'AAAU',
 'AABB',
 'AABVF',
 'AAC',
 'AAC.U',
 'AAC.WS',
 'AACAF',
 'AACAY',
 'AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AACS',
 'AACTF',
 'AADI',
 'AADR',
 'A',
 'AA',
 'AAA',
 'AAAIF',
 'AAALF',
 'AAALY',
 'AAAU',
 'AABB',
 'AABVF',
 'AAC',
 'AAC.U',
 'AAC.WS',
 'AACAF',
 'AACAY',
 'AACG',
 'A',
 'AA',
 'AAA',
 'AAAIF',
 'AAALF',
 'AAALY',
 'AAAU',